# Stochastic fluid model with recourse

In [ ]:
from numpy import *
import random
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
import time

In [ ]:
#record the optimal draining time under differnet collecting period
def opt_tr(range_begin,range_end,model,eps):
    t_range = range(range_begin,range_end,2)
    opt_st1 = np.zeros(len(range(range_begin,range_end,2)))
    for i in range(len(range(range_begin,range_end,2))):
        opt_st1[i] = ms_2_update(eps,t_range[i])
    return opt_st1

In [ ]:
# function use to record the heristic model computing performence
def running_time_record_heristic(model,eps,t,w1):
    #input model name and the tolerence level
    start = time.time()
    model(eps,w1,t)
    end = time.time()
    return str(end-start)

In [ ]:
# function use to record the model computing performence(without initialization)
def running_time_record_heristic(model,eps,time):
    #input model name and the tolerence level
    start = time.time()
    model(eps,time)
    end = time.time()
    return str(end-start)

In [ ]:
#function use to plot the bias trade line
def plot_diff():
    figure3=plt.figure(1)
    diff=np.zeros(10)
    half_widen=np.zeros(10)
    for i in range(0,10):
        diff[i]=(result[i]-result[0])/result[0]
        half_widen[i]=0.01*i
    plt.xlabel("half widen difference of random parameter")
    plt.ylabel("bias")
    plt.plot(half_widen,diff)

## Stochastic fluid model with recourse(without initialization)

In [ ]:
#Now w is unkown and I calculate the draining time under the first stage's decision. if t<t_r, which means all fluid is drain before t_r,
#f_1 is used to calculate theta and gradient. if t>t_r, f_2 will be apllied.
def ms_without_initialization(eps,time):
    t = time
    w = np.zeros(scenario)
    ms_2 = Model("ms_2")
    theta = ms_2.addVar(lb=0,name='theta')
    v = ms_2.addVars(K,lb=0,name='v')    

    ms_2.addConstrs(((quicksum([np.dot(C[i][j],v[j]) for j in range(K)])) <= 1 for i in range(J)),name='cap')
    ms_2.setObjective(theta, GRB.MINIMIZE)
    ms_2.update()
    ms_2.optimize()
    
    z_up = 10000000000
    z_low = 0
    #obj=[]
    k=0
    while z_up - z_low > eps:

        ms_2.optimize()
        ms_2.write("out.lp")
        obj=[]
        # Set objective function
        obj.append(ms_2.objVal)
        v_0=ms_2.getAttr("X")[1:]
        theta_0=ms_2.getAttr("X")[0]
        #print("v_0",v_0)
        #print("theta_0",theta_0)
        
        Gradient = np.zeros((scenario,K))
        Gradient_average = np.zeros(K)
        ms_system = np.zeros(scenario)
            
            
        for s in range(scenario):
            
            ms_f1 = np.zeros(K)
            v_minus_Qalpham = np.zeros(K)
            for i in range(K):
                v_minus_Qalpham[i] = max(0,np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][i])
            #print(v_minus_Qalpham,"v_minus_Qalpham")
            ms_f1 = np.array(Qam)/ (v_minus_Qalpham+0.001)
            ms_system[s] = max(ms_f1[j] for j in range(K))
            #print(ms_system,"ms_system")
            
            if ms_system[s]<=t:#use f_1
                w[s] = 1
                #print("f_1 apply")
                #ms_2.addConstrs((v[j] >= max_Qalpham[s]+0.0001 for j in range(K)),name='vk')
                Gradient_f1 = np.zeros(K)
                max_index = np.argmax(ms_f1)
                    #print("Qam[max_index]",np.transpose(np.array([a - b for a, b in zip(v_0, Qalpham)]))[i][max_index]**2)
                Gradient[s][max_index] = -Qam[max_index]/np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][max_index]**2
                #print("Gradient",Gradient)


                
                
            else:#use f_2
                #print("f_2 apply")
                w[s] = 0
                dual = np.zeros(K)
                ms_2_f2 = Model("ms_2_f2")
                theta_1 = ms_2_f2.addVar(lb=0,name='theta_1')
                x = ms_2_f2.addVars(K,lb=0,name='x')
                ms_2_f2.addConstrs((x[i] <= (v_0[i]*t)/M[i] for i in range(K)),name='cap_f2')
                ms_2_f2.addConstrs((x[i] <= np.transpose(alpha)[s][i]*t + np.transpose(a)[i] + quicksum([np.dot(np.transpose(P)[i][j],x[j]) for j in range(K)]) for i in range(K)),name='remain')
                
                Qaplphatm=np.matmul(Q,(a+np.transpose(alpha)[s]*t))*M
                #print("Qaplphatm",Qaplphatm)
                ms_2_f2.addConstrs((theta_1>=(quicksum(Qaplphatm[i]-M[i]*x[i] for i in range(K) if C[j][i]==1))/(1-(sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)))+t for j in range(J)),name='f2_theta')
                
                ms_2_f2.setObjective(theta_1, GRB.MINIMIZE)
                ms_2_f2.update()
                ms_2_f2.optimize()
                #ms_2_f2.write("out.lp")
                #print("ms_2_f2",ms_2_f2.getAttr("X"))
                x_0=ms_2_f2.getAttr("X")[1:]
                theta_1=ms_2_f2.getAttr("X")[0]
                
                cons_f2=ms_2_f2.getConstrs()
                for i in range(K):                 
                    dual[i] = cons_f2[i].getAttr("Pi")*t/M[i]
                Gradient[s] = t*dual
                #print(dual,"dual")
                
                ms_f2 = np.zeros(J)
                for j in range(J):
                    ms_f2[j] =  (sum(Qaplphatm[i]-M[i]*x_0[i] for i in range(K) if C[j][i]==1))/(1-sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)) + t
                 
                ms_system[s] = max(ms_f2[j] for j in range(J))
        ms_average = sum(ms_system)/scenario        
        #print("ms_system",ms_system)
        
        z_low = theta_0
        if z_up > ms_average:
            z_up = ms_average
        else:
            z_up = z_up
        #print("z_up",z_up)
        #print("z_low",z_low)
        
        Gradient_average = np.sum(Gradient,axis=0)/scenario
        #print("Gradient",Gradient)
        
        #print("Gradient_average",Gradient_average)
                #print("dot",np.dot(Gradient_average,v_0))  
    
        ms_2.addConstr((theta>=ms_average-np.dot(Gradient_average,v_0)+quicksum([np.dot(Gradient_average[j],v[j]) for j in range(K)])),name='feasible_cut')
        ms_2.update()
       
    else:
        ms_2.optimize()
        ms_2.printAttr('X')
        print(w,"w")
    return ms_2.getAttr("X")[0]

## heristic method

In [ ]:
scenario=2
K=4
J=2
alpha1=[0.1,0.3, 0.6,0.9]

#alpha1=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
alpha=np.transpose([[i,0.0,0.0,0.0] for i in alpha1])
a=np.array([10.0, 10.0, 10.0, 10.0])
C=[[1.0,0.0,1.0,0.0],
   [0.0,1.0,0.0,1.0]]
P=[[0.0,1.0,0.0,0.0],
    [0.0,0.0,1.0,0.0],
    [0.0,0.0,0.0,1.0],
    [0.0,0.0,0.0,0.0]]
I=np.eye(K)
M=[0.3,0.3,0.5,0.2]
Q=np.linalg.inv(I-np.transpose(P))
Qalpha=np.matmul(Q,alpha)
Qalpham=[Qalpha[i]*M[i] for i in range(K)]
max_Qalpham = np.amax(Qalpham, 1)
Qa=np.matmul(Q,a)
Qam=[Qa[i]*M[i] for i in range(K)]

In [ ]:
def ms_heristic(eps,w1,t):
    w=w1
    ms_2 = Model("ms_2")
    theta = ms_2.addVar(lb=0,name='theta')
    v = ms_2.addVars(K,lb=0,name='v')    
    for s in range(scenario):
        if w[s]==1:
            ms_2.addConstrs((v[j] >= np.transpose(Qalpham)[s][j]+0.0001 for j in range(K)),name='vk')
    ms_2.addConstrs(((quicksum([np.dot(C[i][j],v[j]) for j in range(K)])) <= 1 for i in range(J)),name='cap')
    ms_2.setObjective(theta, GRB.MINIMIZE)
    ms_2.update()
    ms_2.optimize()
    
    
    z_up = 10000000000
    z_low = 0
    #obj=[]
    k=0
    while z_up - z_low > eps:

        ms_2.optimize()
        ms_2.write("out.lp")
        obj=[]
        # Set objective function
        obj.append(ms_2.objVal)
        #n=len(obj)
        #print("obj",obj)
        v_0=ms_2.getAttr("X")[1:]
        theta_0=ms_2.getAttr("X")[0]
        #print("v_0",v_0)
        #print("theta_0",theta_0)
        
        Gradient = np.zeros((scenario,K))
        Gradient_average = np.zeros(K)
        ms_system = np.zeros(scenario)
            
            
        for s in range(scenario):
            
            ms_f1 = np.zeros(K)
            v_minus_Qalpham = np.zeros(K)
            for i in range(K):
                v_minus_Qalpham[i] = max(0,np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][i])
            #print(v_minus_Qalpham,"v_minus_Qalpham")
            ms_f1 = np.array(Qam)/ (v_minus_Qalpham+0.001)
            ms_system[s] = max(ms_f1[j] for j in range(K))
            if ms_system[s]<=t:#use f_1
                w[s] = 1
                #print("f_1 apply")
                #ms_2.addConstrs((v[j] >= max_Qalpham[s]+0.0001 for j in range(K)),name='vk')
                Gradient_f1 = np.zeros(K)
                max_index = np.argmax(ms_f1)
                    #print("Qam[max_index]",np.transpose(np.array([a - b for a, b in zip(v_0, Qalpham)]))[i][max_index]**2)
                Gradient[s][max_index] = -Qam[max_index]/np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][max_index]**2
                #print("Gradient",Gradient)


                
                
            else:#use f_2
                #print("f_2 apply")
                w[s] = 0
                dual = np.zeros(K)
                ms_2_f2 = Model("ms_2_f2")
                theta_1 = ms_2_f2.addVar(lb=0,name='theta_1')
                x = ms_2_f2.addVars(K,lb=0,name='x')
                ms_2_f2.addConstrs((x[i] <= (v_0[i]*t)/M[i] for i in range(K)),name='cap_f2')
                ms_2_f2.addConstrs((x[i] <= np.transpose(alpha)[s][i]*t + np.transpose(a)[i] + quicksum([np.dot(np.transpose(P)[i][j],x[j]) for j in range(K)]) for i in range(K)),name='remain')
                
                Qaplphatm=np.matmul(Q,(a+np.transpose(alpha)[s]*t))*M
                #print("Qaplphatm",Qaplphatm)
                ms_2_f2.addConstrs((theta_1>=(quicksum(Qaplphatm[i]-M[i]*x[i] for i in range(K) if C[j][i]==1))/(1-(sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)))+t for j in range(J)),name='f2_theta')
                
                ms_2_f2.setObjective(theta_1, GRB.MINIMIZE)
                ms_2_f2.update()
                ms_2_f2.optimize()
                #ms_2_f2.write("out.lp")
                #print("ms_2_f2",ms_2_f2.getAttr("X"))
                x_0=ms_2_f2.getAttr("X")[1:]
                theta_1=ms_2_f2.getAttr("X")[0]
                
                cons_f2=ms_2_f2.getConstrs()
                for i in range(K):                 
                    dual[i] = cons_f2[i].getAttr("Pi")*t/M[i]
                Gradient[s] = t*dual
                #print(dual,"dual")
                
                ms_f2 = np.zeros(J)
                for j in range(J):
                    ms_f2[j] =  (sum(Qaplphatm[i]-M[i]*x_0[i] for i in range(K) if C[j][i]==1))/(1-sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)) + t
                 
                ms_system[s] = max(ms_f2[j] for j in range(J))
        ms_average = sum(ms_system)/scenario        
        #print("ms_system",ms_system)
        
        z_low = theta_0
        if z_up > ms_average:
            z_up = ms_average
        else:
            z_up = z_up
        #print("z_up",z_up)
        #print("z_low",z_low)
        
        Gradient_average = np.sum(Gradient,axis=0)/scenario
        #print("Gradient",Gradient)
        
        #print("Gradient_average",Gradient_average)
                #print("dot",np.dot(Gradient_average,v_0))  
    
        ms_2.addConstr((theta>=ms_average-np.dot(Gradient_average,v_0)+quicksum([np.dot(Gradient_average[j],v[j]) for j in range(K)])),name='feasible_cut')
        ms_2.update()
       
    else:
        ms_2.optimize()
        ms_2.printAttr('X')
        print(w,"w")
    return ms_2.getAttr("X")[0]
    

## deterministic VS random (random alpha, 2 scenarios)

In [ ]:
scenario=2
K=4
J=2    
a=np.array([10.0, 10.0, 10.0, 10.0])
C=[[1.0,0.0,1.0,0.0],
    [0.0,1.0,0.0,1.0]]
P=[[0.0,1.0,0.0,0.0],
    [0.0,0.0,1.0,0.0],
    [0.0,0.0,0.0,1.0],
    [0.0,0.0,0.0,0.0]]
I=np.eye(K)
M = [0.3,0.3,0.5,0.2]

Q=np.linalg.inv(I-np.transpose(P))
Qa=np.matmul(Q,a)
Qam=[Qa[i]*M[i] for i in range(K)]

t = 0
result=np.zeros(10)
for d in range(10):
    alpha1 = [0.5-0.01*d,0.5+0.01*d]
    alpha=np.transpose([[i,0.0,0.0,0.0] for i in alpha1])
    Qalpha=np.matmul(Q,alpha)
    Qalpham=[Qalpha[i]*M[i] for i in range(K)]
    w = np.zeros(scenario)
    ms_2 = Model("ms_2")
    theta = ms_2.addVar(lb=0,name='theta')
    v = ms_2.addVars(K,lb=0,name='v')    

    ms_2.addConstrs(((quicksum([np.dot(C[i][j],v[j]) for j in range(K)])) <= 1 for i in range(J)),name='cap')
    ms_2.setObjective(theta, GRB.MINIMIZE)
    ms_2.update()
    ms_2.optimize()
    
    z_up = 10000000000
    z_low = 0
    #obj=[]
    k=0
    while z_up - z_low > 0.1:

        ms_2.optimize()
        ms_2.write("out.lp")
        obj=[]
        # Set objective function
        obj.append(ms_2.objVal)
        v_0=ms_2.getAttr("X")[1:]
        theta_0=ms_2.getAttr("X")[0]
        #print("v_0",v_0)
        #print("theta_0",theta_0)
        
        Gradient = np.zeros((scenario,K))
        Gradient_average = np.zeros(K)
        ms_system = np.zeros(scenario)
            
            
        for s in range(scenario):
            
            ms_f1 = np.zeros(K)
            v_minus_Qalpham = np.zeros(K)
            for i in range(K):
                v_minus_Qalpham[i] = max(0,np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][i])
            #print(v_minus_Qalpham,"v_minus_Qalpham")
            ms_f1 = np.array(Qam)/ (v_minus_Qalpham+0.001)
            ms_system[s] = max(ms_f1[j] for j in range(K))
            #print(ms_system,"ms_system")
            
            if ms_system[s]<=t:#use f_1
                w[s] = 1
                #print("f_1 apply")
                #ms_2.addConstrs((v[j] >= max_Qalpham[s]+0.0001 for j in range(K)),name='vk')
                Gradient_f1 = np.zeros(K)
                max_index = np.argmax(ms_f1)
                    #print("Qam[max_index]",np.transpose(np.array([a - b for a, b in zip(v_0, Qalpham)]))[i][max_index]**2)
                Gradient[s][max_index] = -Qam[max_index]/np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[s][max_index]**2
                #print("Gradient",Gradient)


                
                
            else:#use f_2
                #print("f_2 apply")
                w[s] = 0
                dual = np.zeros(K)
                ms_2_f2 = Model("ms_2_f2")
                theta_1 = ms_2_f2.addVar(lb=0,name='theta_1')
                x = ms_2_f2.addVars(K,lb=0,name='x')
                ms_2_f2.addConstrs((x[i] <= (v_0[i]*t)/M[i] for i in range(K)),name='cap_f2')
                ms_2_f2.addConstrs((x[i] <= np.transpose(alpha)[s][i]*t + np.transpose(a)[i] + quicksum([np.dot(np.transpose(P)[i][j],x[j]) for j in range(K)]) for i in range(K)),name='remain')
                
                Qaplphatm=np.matmul(Q,(a+np.transpose(alpha)[s]*t))*M
                #print("Qaplphatm",Qaplphatm)
                ms_2_f2.addConstrs((theta_1>=(quicksum(Qaplphatm[i]-M[i]*x[i] for i in range(K) if C[j][i]==1))/(1-(sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)))+t for j in range(J)),name='f2_theta')
                
                ms_2_f2.setObjective(theta_1, GRB.MINIMIZE)
                ms_2_f2.update()
                ms_2_f2.optimize()
                #ms_2_f2.write("out.lp")
                #print("ms_2_f2",ms_2_f2.getAttr("X"))
                x_0=ms_2_f2.getAttr("X")[1:]
                theta_1=ms_2_f2.getAttr("X")[0]
                
                cons_f2=ms_2_f2.getConstrs()
                for i in range(K):                 
                    dual[i] = cons_f2[i].getAttr("Pi")*t/M[i]
                Gradient[s] = t*dual
                #print(dual,"dual")
                
                ms_f2 = np.zeros(J)
                for j in range(J):
                    ms_f2[j] =  (sum(Qaplphatm[i]-M[i]*x_0[i] for i in range(K) if C[j][i]==1))/(1-sum(np.transpose(Qalpham)[s][i] for i in range(K) if C[j][i]==1)) + t
                 
                ms_system[s] = max(ms_f2[j] for j in range(J))
        ms_average = sum(ms_system)/scenario        
        #print("ms_system",ms_system)
        
        z_low = theta_0
        if z_up > ms_average:
            z_up = ms_average
        else:
            z_up = z_up
        #print("z_up",z_up)
        #print("z_low",z_low)
        
        Gradient_average = np.sum(Gradient,axis=0)/scenario
        #print("Gradient",Gradient)
        
        #print("Gradient_average",Gradient_average)
                #print("dot",np.dot(Gradient_average,v_0))  
    
        ms_2.addConstr((theta>=ms_average-np.dot(Gradient_average,v_0)+quicksum([np.dot(Gradient_average[j],v[j]) for j in range(K)])),name='feasible_cut')
        ms_2.update()
       
    else:
        ms_2.optimize()
        ms_2.printAttr('X')
        #print(w,"w")


   
    result[d]=ms_2.getAttr("X")[0]
result
plot_diff()

## deterministic VS random (random a, 2 scenarios)

In [ ]:
scenario=2
K=4
J=2    
t= 20
C=[[1.0,0.0,1.0,0.0],
    [0.0,1.0,0.0,1.0]]
P=[[0.0,1.0,0.0,0.0],
    [0.0,0.0,1.0,0.0],
    [0.0,0.0,0.0,1.0],
    [0.0,0.0,0.0,0.0]]
I=np.eye(K)
M = [0.3,0.3,0.5,0.2]
alpha=np.transpose([0.5,0.0,0.0,0.0])
Q=np.linalg.inv(I-np.transpose(P))
Qalpha=np.matmul(Q,alpha)
Qalpham=Qalpha*M

result=np.zeros(10)
for d in range(10):
    a1 = [10-0.5*d,10+0.5*d]
    a = np.transpose([[10,a1[i],10,10] for i in range(2)])
    Qa=np.matmul(Q,a)
    Qam=[Qa[i]*M[i] for i in range(K)]
    w = np.zeros(scenario)
    ms_2 = Model("ms_2")
    theta = ms_2.addVar(lb=0,name='theta')
    v = ms_2.addVars(K,lb=0,name='v')    

    ms_2.addConstrs(((quicksum([np.dot(C[i][j],v[j]) for j in range(K)])) <= 1 for i in range(J)),name='cap')
    ms_2.setObjective(theta, GRB.MINIMIZE)
    ms_2.update()
    ms_2.optimize()
    
    z_up = 10000000000
    z_low = 0
    #obj=[]
    k=0
    while z_up - z_low > 0.1:

        ms_2.optimize()
        ms_2.write("out.lp")
        obj=[]
        # Set objective function
        obj.append(ms_2.objVal)
        v_0=ms_2.getAttr("X")[1:]
        theta_0=ms_2.getAttr("X")[0]
        #print("v_0",v_0)
        #print("theta_0",theta_0)
        
        Gradient = np.zeros((scenario,K))
        Gradient_average = np.zeros(K)
        ms_system = np.zeros(scenario)
            
            
        for s in range(scenario):
            
            ms_f1 = np.zeros(K)
            v_minus_Qalpham = np.zeros(K)
            for i in range(K):
                v_minus_Qalpham[i] = max(0,np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[i])
            #print(v_minus_Qalpham,"v_minus_Qalpham")
            ms_f1 = np.transpose(Qam)[s]/ (v_minus_Qalpham+0.001)
            #ms[i] =  np.transpose(Qam)[i]/ np.transpose(np.array([a - b for a, b in zip(v_0, Qalpham)]))
            ms_system[s] = max(ms_f1[j] for j in range(K))
            #print(ms_system,"ms_system")
            
            if ms_system[s]<=t:#use f_1
                w[s] = 1
                #print("f_1 apply")
                #ms_2.addConstrs((v[j] >= max_Qalpham[s]+0.0001 for j in range(K)),name='vk')
                Gradient_f1 = np.zeros(K)
                max_index = np.argmax(ms_f1)
                    #print("Qam[max_index]",np.transpose(np.array([a - b for a, b in zip(v_0, Qalpham)]))[i][max_index]**2)
                Gradient[s][max_index] = -np.transpose(Qam)[s][max_index]/np.transpose(np.array([m - n for m, n in zip(v_0, Qalpham)]))[max_index]**2

                #print("Gradient",Gradient)


                
                
            else:#use f_2
                #print("f_2 apply")
                w[s] = 0
                dual = np.zeros(K)
                ms_2_f2 = Model("ms_2_f2")
                theta_1 = ms_2_f2.addVar(lb=0,name='theta_1')
                x = ms_2_f2.addVars(K,lb=0,name='x')
                ms_2_f2.addConstrs((x[i] <= (v_0[i]*t)/M[i] for i in range(K)),name='cap_f2')
                ms_2_f2.addConstrs((x[i] <= np.transpose(alpha)[i]*t + np.transpose(a)[s][i] + quicksum([np.dot(np.transpose(P)[i][j],x[j]) for j in range(K)]) for i in range(K)),name='remain')
                
                Qaplphatm=np.matmul(Q,(np.transpose(a)[0]+alpha*t)*M)
                #print("Qaplphatm",Qaplphatm)
                ms_2_f2.addConstrs((theta_1>=(quicksum(Qaplphatm[i]-M[i]*x[i] for i in range(K) if C[j][i]==1))/(1-(sum(np.transpose(Qalpham)[i] for i in range(K) if C[j][i]==1)))+t for j in range(J)),name='f2_theta')
                
                ms_2_f2.setObjective(theta_1, GRB.MINIMIZE)
                ms_2_f2.update()
                ms_2_f2.optimize()
                #ms_2_f2.write("out.lp")
                #print("ms_2_f2",ms_2_f2.getAttr("X"))
                x_0=ms_2_f2.getAttr("X")[1:]
                theta_1=ms_2_f2.getAttr("X")[0]
                
                cons_f2=ms_2_f2.getConstrs()
                for i in range(K):                 
                    dual[i] = cons_f2[i].getAttr("Pi")*t/M[i]
                Gradient[s] = t*dual
                #print(dual,"dual")
                
                ms_f2 = np.zeros(J)
                for j in range(J):
                    ms_f2[j] =  (sum(Qaplphatm[i]-M[i]*x_0[i] for i in range(K) if C[j][i]==1))/(1-sum(np.transpose(Qalpham)[i] for i in range(K) if C[j][i]==1)) + t
                 
                ms_system[s] = max(ms_f2[j] for j in range(J))
        ms_average = sum(ms_system)/scenario        
        #print("ms_system",ms_system)
        
        z_low = theta_0
        if z_up > ms_average:
            z_up = ms_average
        else:
            z_up = z_up
        #print("z_up",z_up)
        #print("z_low",z_low)
        
        Gradient_average = np.sum(Gradient,axis=0)/scenario
        #print("Gradient",Gradient)
        
        #print("Gradient_average",Gradient_average)
                #print("dot",np.dot(Gradient_average,v_0))  
    
        ms_2.addConstr((theta>=ms_average-np.dot(Gradient_average,v_0)+quicksum([np.dot(Gradient_average[j],v[j]) for j in range(K)])),name='feasible_cut')
        ms_2.update()
       
    else:
        ms_2.optimize()
        ms_2.printAttr('X')
        #print(w,"w")


   
    result[d]=ms_2.getAttr("X")[0]
result
plot_diff()